In [114]:
import pandas as pd
import numpy as np
import copy
# from sklearn.model_selection import

load data

In [115]:
# full_data分为 测试数据和验证数据 test_data，vali_data
df = pd.read_csv('./data/full_data.csv', encoding='utf-16', sep='\t')
df.drop(columns=['Unnamed: 0'], inplace=True)
num = int(len(df) * 0.5)
test_data = df.iloc[:num,:]
# test_data.to_csv('./data/test_data.csv',encoding='utf-16', sep='\t')
vali_data = df.iloc[num:, :]
# vali_data = vali_data.to_csv('./data/vali_data.csv', encoding='utf-16', sep='\t')

In [116]:
# 用 test_data 训练
data = copy.deepcopy(test_data)
# data.head()
data.shape
col_names = data.columns.to_list()
colidx_y = data.columns.get_loc('KLD热风风门开度（%）d')
corr = data.corr()

split data into trainintg sets, test sets, cross-validate sets

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [117]:
# data = shuffle(data, n_samples=len(data))

y = data.loc[:,['KLD热风风门开度（%）d']]
x = data.drop(columns=['KLD热风风门开度（%）d'])
x.shape, y.shape

((20739, 39), (20739, 1))

In [118]:
x_train, x_validate_test, y_train, y_validate_test = train_test_split(x, y, test_size=0.2, random_state = 42, shuffle=True)
x_validate, x_test, y_validate, y_test = train_test_split(x_validate_test, y_validate_test, test_size=0.5, random_state=42, shuffle=True)

In [25]:
x_train.shape, x_validate.shape, x_test.shape,y_train.shape

((16591, 39), (2074, 39), (2074, 39), (16591, 1))

feature selection

In [101]:
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr
from sklearn.feature_selection import f_regression, mutual_info_regression

from sklearn .feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [124]:
# x_train = np.array(x_train)
# y_train = np.array(y_train)
# r,p = pearsonr(x_train, y_train)
x_new = SelectKBest(mutual_info_regression, k=15).fit(x_train, y_train)
x_new_c = x_new.get_support(indices=True)
x_train.iloc[:,x_new_c].columns

e:\code\china_tobacco\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['时间', 'KLD1区蒸汽薄膜阀开度（%）d', 'KLD一次减压后蒸汽压力（bar）d', 'KLD烘丝机管道风速差压传感器读数d',
       'KLD烘后温度（℃）d', 'KLD热风蒸汽薄膜阀开度（%）d', 'KLD热风进筒温度（℃）d', 'KLD除水量（l/h）d',
       'PC-37排潮风机负压值d', 'ZC-36除尘风机负压值d', '冷却温度（℃）d', '制丝高压蒸汽DN200干度-薄板D',
       '烘丝筒1区筒壁温度设定', '除尘房排潮风门开度d', '风选冷却排潮负压（mbar）d'],
      dtype='object')

In [121]:
x_new.scores_

(array([0.17215724, 0.01244528, 0.16759309, 0.04041548, 0.12608986,
        0.06515049, 0.16981249, 0.05892385, 0.11859043, 0.01714471,
        0.09278146, 0.15028936, 0.08923075, 0.0257294 , 0.12140335,
        0.00483457, 0.0029375 , 0.13587469, 0.10733172, 0.03320424,
        0.08203917, 0.0040546 , 0.04195936, 0.02920418, 0.00307371,
        0.00508508, 0.15130775, 0.03079796, 0.01714894, 0.03612184,
        0.        , 0.03124018, 0.00781395, 0.00189467, 0.00412953,
        0.13723082, 0.00309681, 0.02838441, 0.08261134]),
 None)

In [ ]:
['时间', 'KLD1区蒸汽薄膜阀开度（%）d', 'KLDD烘丝分汽缸压力', 'KLD一次减压后蒸汽压力（bar）d',
       'KLD排潮风门开度（%）d', 'KLD烘丝机管道风速差压传感器读数d', 'KLD烘后水分（%）d', 'KLD烘后温度（℃）d',
       'KLD热风蒸汽薄膜阀开度（%）d', 'KLD热风进筒温度（℃）d', 'KLD除水量（l/h）d', 'PC-37排潮风机负压值d',
       'ZC-36除尘风机负压值d', '冷却温度（℃）d', '制丝高压蒸汽DN200干度-薄板D', '制丝高压蒸汽DN200瞬时流量-薄板D',
       '烘丝筒1区筒壁温度设定', '烘前水分（%）d', '除尘房排潮风门开度d', '风选冷却排潮负压（mbar）d']

In [102]:
# 递归特征消除
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=20)
rfe.fit_transform(x_train, y_train)
new_xc = rfe.support_
new_xc = rfe.ranking_

e:\code\china_tobacco\venv\lib\site-packages\sklearn\feature_selection\_rfe.py:299: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
e:\code\china_tobacco\venv\lib\site-packages\sklearn\feature_selection\_rfe.py:299: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
e:\code\china_tobacco\venv\lib\site-packages\sklearn\feature_selection\_rfe.py:299: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
e:\code\china_tobacco\venv\lib\site-packages\sklearn\feature_selection\_rfe.py:299: DataConversionWarning: A column-vector y was passed 

In [125]:
list = ['时间', 'KLD1区蒸汽薄膜阀开度（%）d', 'KLD一次减压后蒸汽压力（bar）d', 'KLD排潮风门开度（%）d',
       'KLD烘丝机管道风速差压传感器读数d', 'KLD烘后水分（%）d', 'KLD烘后温度（℃）d', 'KLD热风蒸汽薄膜阀开度（%）d',
       'KLD热风进筒温度（℃）d', 'KLD除水量（l/h）d', 'PC-37排潮风机负压值d', 'ZC-36除尘风机负压值d',
       '冷却温度（℃）d', '制丝高压蒸汽DN200干度-薄板D', '制丝高压蒸汽DN200瞬时流量-薄板D', '烘丝筒1区筒壁温度设定',
       '烘前水分（%）d', '筒体电机电流d', '除尘房排潮风门开度d', '风选冷却排潮负压（mbar）d',
       
       'KLD热风温度（℃）d',

       'KLD热风温度（℃）d','烘前叶丝流量（kg/h）d','烘前喂料机提升带速度',
       'KLDD烘丝分汽缸压力','制丝高压蒸汽DN200压力-薄板D','排潮风机电机电流d','气锁电机电流d','膨胀轮电机电流d','锅炉分汽缸压力-薄板D','风选冷却排潮开度（%）d']
len(list)

31

In [ ]:
col = ['除尘房除尘风机频率d', '除尘房排潮风机频率(Hz)d', '排潮风机电机频率d', 'ZC-35除尘风机运行频率d','PC-37排潮风机运行频率d']

In [103]:
index = np.where(new_xc==1)
list = []
for i in index[0]:
    print(col_names[i])

时间
KLD1区蒸汽薄膜阀开度（%）d
KLD排潮风门开度（%）d
KLD烘丝机管道风速差压传感器读数d
KLD烘后水分（%）d
KLD烘后温度（℃）d
KLD热风温度（℃）d
KLD热风进筒温度（℃）d
KLD除水量（l/h）d
PC-37排潮风机负压值d
ZC-36除尘风机负压值d
冷却温度（℃）d
制丝高压蒸汽DN200干度-薄板D
制丝高压蒸汽DN200温度-薄板D
烘丝筒1区筒壁温度设定
烘前叶丝流量（kg/h）d
烘前喂料机提升带速度
筒体电机电流d
除尘房排潮风门开度d
风选冷却排潮负压（mbar）d


In [92]:
index = np.where(new_xc==1)
list = []
for i in index[0]:
    print(col_names[i])

KLD1区蒸汽薄膜阀开度（%）d
KLDD烘丝分汽缸压力
KLD一次减压后蒸汽压力（bar）d
KLD排潮风门开度（%）d
KLD烘丝机管道风速差压传感器读数d
KLD烘后水分（%）d
KLD热风温度（℃）d
KLD热风蒸汽薄膜阀开度（%）d
KLD热风进筒温度（℃）d
PC-37排潮风机负压值d
ZC-36除尘风机负压值d
冷却温度（℃）d
制丝高压蒸汽DN200压力-薄板D
制丝高压蒸汽DN200干度-薄板D
排潮风机电机电流d
气锁电机电流d
膨胀轮电机电流d
锅炉分汽缸压力-薄板D
除尘房排潮风门开度d
风选冷却排潮开度（%）d


In [111]:
abs(corr['KLD热风风门开度（%）d']).sort_values(ascending=False)

KLD热风风门开度（%）d          1.000000
KLD烘丝机管道风速差压传感器读数d     0.890765
除尘房排潮风门开度d             0.741331
ZC-36除尘风机负压值d          0.586022
KLD1区蒸汽薄膜阀开度（%）d       0.434738
KLD热风进筒温度（℃）d          0.378082
冷却温度（℃）d               0.331766
风选冷却排潮负压（mbar）d        0.278328
KLD烘后温度（℃）d            0.240865
PC-37排潮风机负压值d          0.226142
烘丝筒1区筒壁温度设定            0.216435
KLD一次减压后蒸汽压力（bar）d     0.205111
时间                     0.152828
KLD热风温度（℃）d            0.101535
KLDD烘丝分汽缸压力            0.062784
排潮风机电机电流d              0.056167
KLD热风蒸汽薄膜阀开度（%）d       0.056042
制丝高压蒸汽DN200干度-薄板D      0.042152
筒体电机电流d                0.040623
膨胀轮电机电流d               0.034199
烘前叶丝流量（kg/h）d          0.033796
制丝高压蒸汽DN200瞬时流量-薄板D    0.032185
KLD除水量（l/h）d           0.027081
PC-19排潮风机负压值           0.020214
制丝高压蒸汽DN200压力-薄板D      0.019387
KLD烘后水分（%）d            0.018700
锅炉分汽缸压力-薄板D            0.015807
烘前喂料机提升带速度             0.008536
烘前水分（%）d               0.006098
风选冷却排潮开度（%）d           0.006012
2号切丝机运行                0.005513
制丝高压蒸汽DN

In [113]:
abs(corr['KLD烘丝机管道风速差压传感器读数d']).sort_values(ascending=False)
# 除尘房排潮风门开度d

KLD烘丝机管道风速差压传感器读数d     1.000000
KLD热风风门开度（%）d          0.890765
除尘房排潮风门开度d             0.723192
ZC-36除尘风机负压值d          0.588616
KLD热风进筒温度（℃）d          0.557578
冷却温度（℃）d               0.309354
KLD1区蒸汽薄膜阀开度（%）d       0.273314
风选冷却排潮负压（mbar）d        0.262301
PC-37排潮风机负压值d          0.197469
KLD一次减压后蒸汽压力（bar）d     0.191187
KLD除水量（l/h）d           0.181896
KLD烘后温度（℃）d            0.172017
KLD热风温度（℃）d            0.142083
时间                     0.138826
烘前叶丝流量（kg/h）d          0.132611
烘丝筒1区筒壁温度设定            0.127344
KLD烘后水分（%）d            0.096855
烘前水分（%）d               0.054781
KLD热风蒸汽薄膜阀开度（%）d       0.053068
KLDD烘丝分汽缸压力            0.047167
KLD排潮风门开度（%）d          0.039134
制丝高压蒸汽DN200干度-薄板D      0.032958
膨胀轮电机电流d               0.027878
制丝高压蒸汽DN200瞬时流量-薄板D    0.022721
排潮风机电机电流d              0.018464
PC-19排潮风机负压值           0.016567
锅炉分汽缸压力-薄板D            0.013538
制丝高压蒸汽DN200压力-薄板D      0.012142
2号切丝机运行                0.010096
烘前喂料机提升带速度             0.010033
热风电机电流d                0.007329
风选冷却排潮开度